In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcl
import pandas as pd

dataSell = pd.read_csv("/content/drive/MyDrive/NUCLIO/Nuclio-Capstone/data_dsmarket/data_dsmarket/item_sales.csv")


In [ ]:
# función para relacionar días semana
def days2week(days):
   weekStart = 201105
   dayStart = "Saturday"
   dayStartInt = 0

   lastWeekYear = {}
   lastWeekYear[2011] = 52
   lastWeekYear[2012] = 53
   lastWeekYear[2013] = 52
   lastWeekYear[2014] = 52
   lastWeekYear[2015] = 52
   lastWeekYear[2016] = 52

   weekDays = 7

   week = weekStart
   dayInt = dayStartInt

   days2week = {}
   nDays = days
   for i in range(1, nDays + 1):
     days2week[i] = week
     dayInt += 1
     #print(dayInt)
     if dayInt == 7:
       dayInt = 0
       #print(dayInt)
       week += 1

       currentYear = int(str(week)[0:4])
       currentWeek = int(str(week)[4:])

       if (lastWeekYear[currentYear] == currentWeek):
         week = (currentYear + 1) * 100 


   return days2week

d2Week=days2week(1913)

In [ ]:
dataCol = dataSell.columns

dayCol = [col for col in dataCol if 'd_' in col]
noDayCol = [col for col in dataCol if not 'd_' in col]
noDayCol

['id', 'item', 'category', 'department', 'store', 'store_code', 'region']

In [ ]:
#Pasamos días de col a filas
dataSell = dataSell.melt(id_vars = noDayCol,
                 var_name = "day",
                 value_name = "sell")

In [ ]:
# Para todas las tiendas agrupamos por item y día
dataSell = dataSell.groupby(['category', 'day']).sum().reset_index()

In [ ]:
# Pasamos los días a numéricos
dataSell['day'] = dataSell['day'].str[2:]
dataSell

,department,day,sell
0,ACCESORIES_1,1,3610
1,ACCESORIES_1,10,2514
2,ACCESORIES_1,100,2714
3,ACCESORIES_1,1000,2876
4,ACCESORIES_1,1001,3101
...,...,...,...
13386,SUPERMARKET_3,995,21269
13387,SUPERMARKET_3,996,22381
13388,SUPERMARKET_3,997,16036
13389,SUPERMARKET_3,998,14170


In [ ]:
# Relacionamos días-semana
weekCol = []
for row in dataSell["day"]:
  weekCol.append(d2Week[int(row)])

dataSell["week"] = weekCol

In [ ]:
dataSell = dataSell.groupby(['category', 'week']).sum().reset_index()

In [ ]:
#Pintamos por departamentos
x = dataSell['week'].to_numpy()
y = dataSell['sell'].to_numpy()
z = dataSell['category'].to_numpy()

import numpy as np
zUniq = np.unique(z)

colorLegend = {}
fig = plt.figure(figsize=(25,10))
ax = plt.gca()
for i in range(len(zUniq)):
  x = dataSell.loc[dataSell['category'] == zUniq[i], 'week'].to_numpy()
  y = dataSell.loc[dataSell['category'] == zUniq[i], 'sell'].to_numpy()
  plt.plot(x, y, '-o')
  colorLegend[zUniq[i]] = ax.get_lines()[-1].get_color()
ax.legend(labels = zUniq)